In [0]:
%pip install --upgrade -Uqqq mlflow>=3.0 xgboost optuna uv
%restart_python

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import mlflow
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from mlflow.models import infer_signature
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
from datetime import datetime, timezone, timedelta
import optuna
from functools import partial
from mlflow import MlflowClient
import logging


In [0]:
PATH_BASE_PROCESSADA = 'workspace.processed.base_processada'

In [0]:
# Vamos usar para não poluir demais as saídas do MLFlow

mlflow_logger = logging.getLogger("mlflow")
mlflow_logger.setLevel(logging.ERROR)
print("Nível de log do MLflow ajustado para ERROR. Apenas erros serão exibidos.")

In [0]:
dados = spark.table(PATH_BASE_PROCESSADA).toPandas()

## 1. Configurando o Model Registry

O código abaixo serve para configurar o local onde o MLflow vai registrar e gerenciar os modelos. O parâmetro "databricks-uc" é um alias que indica que o Registry de Modelos do MLflow deve ser integrado ao Unity Catalog do Databricks. Caso não fosse inserido essa linha os modelos ficam no registry local do MLflow ao inserir, os modelos ficam registrados no Unity Catalog.

In [0]:
mlflow.set_registry_uri("databricks-uc")

## 2. EDA

In [0]:
minhas_cores = [
    '#FE3054', # vermelho vibrante
    '#F94EB6', # rosa choque
    '#FFB020', # amarelo ouro
    '#FA781A', # laranja intenso
    '#FEF6C8', # bege claro
    '#06516E', # azul petróleo
    '#06987B', # verde esmeralda
    '#260607'  # vinho quase preto
]

minha_paleta = sns.color_palette(minhas_cores)

sns.palplot(minha_paleta)
plt.title("Minha Paleta Personalizada")
plt.show()

sns.set_palette(minha_paleta)

In [0]:
def plot_roc_auc(y_test, y_scores):
    fpr, tpr, _ = roc_curve(y_test, y_scores)
    roc_auc = auc(fpr, tpr)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.plot(fpr, tpr, color='#FE3054', lw=2, label=f'AUC = {roc_auc:.2f}')
    ax.plot([0, 1], [0, 1], color='gray', linestyle='--') 
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Curva ROC')
    ax.legend(loc='lower right')
    ax.grid()
    
    return fig 


def plot_matriz_confusao(y_true_teste, y_pred_teste, group_names=None,
                         categories='auto', count=True,
                         xyticks=True, sum_stats=True, figsize=None,
                         title=None):

    cf = confusion_matrix(y_true_teste, y_pred_teste)

    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names) == cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = ["{0:0.0f}\n".format(value) for value in cf.flatten()]
    else:
        group_counts = blanks

    box_labels = [f"{v1}{v2}".strip()
                  for v1, v2 in zip(group_labels, group_counts)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0], cf.shape[1])

    if sum_stats:
        accuracy = accuracy_score(y_true_teste, y_pred_teste)
        precision = precision_score(y_true_teste, y_pred_teste)
        recall = recall_score(y_true_teste, y_pred_teste)
        f1_score_metric = f1_score(y_true_teste, y_pred_teste)

        stats_text = "Acurácia = {:0.3f}\nPrecisão = {:0.3f}\nRecall = {:0.3f}\nF1 Score = {:0.3f}".format(
            accuracy, precision, recall, f1_score_metric)
    else:
        stats_text = ""

    if figsize is None:
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks is False:
        categories = False

    fig, ax = plt.subplots(figsize=figsize)
    sns.set(font_scale=1.4)
    sns.heatmap(cf, annot=box_labels, fmt="", cbar=False,
                xticklabels=categories, yticklabels=categories, ax=ax)
    

    ax.text(cf.shape[1] + 0.7, cf.shape[0] / 2.0, stats_text, ha='left', va='center', fontsize=16)

    ax.set_ylabel('Valores verdadeiros', fontsize=17)
    ax.set_xlabel('Valores preditos', fontsize=17)

    if title:
        ax.set_title(title, fontsize=20, pad=20)
    
    fig.tight_layout(rect=[0, 0, 0.85, 1]) 
    return fig


def heatmap_corr(df: pd.DataFrame, figsize: tuple = (8, 6)):
    """
    Gera um heatmap de correlação triangular para um DataFrame.

    Argumentos:
        df (pd.DataFrame): O DataFrame de entrada com dados numéricos.
        figsize (tuple, optional): O tamanho da figura (largura, altura). Padrão é (8, 6).
    """

    corr = df.corr(numeric_only=True)
    
    mask = np.triu(np.ones_like(corr, dtype=bool))

    plt.figure(figsize=figsize)

    ax = sns.heatmap(corr,
                mask=mask,
                annot=True,
                fmt=".2f",
                xticklabels=corr.columns.values,
                yticklabels=corr.columns.values)
    
    ax.set_xticklabels(ax.get_xticklabels(), ha='right')
    
    plt.title('Heatmap de Correlação')
    plt.show()
    

def plotar_outliers(X: pd.DataFrame, n_cols: int = 3) -> plt.Figure:
    """
    Cria uma grade de boxplots para detectar outliers em cada variável.

    Args:
        X (pd.DataFrame): DataFrame contendo as variáveis.
        n_cols (int): Número de colunas no layout da grade.

    Returns:
        plt.Figure: Objeto Figure do matplotlib contendo os gráficos de outliers.
    """
    features = X.columns
    n_features = len(features)
    n_rows = (n_features + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows))
    axes = axes.flatten() if n_rows * n_cols > 1 else [axes]
    
    for i, feature in enumerate(features):
        if i < len(axes):
            ax = axes[i]
            sns.boxplot(x=X[feature], ax=ax)
            ax.set_title(f'{feature}')
            ax.set_xlabel(feature)

    for i in range(n_features, len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    fig.suptitle('Detecção de outliers para as variáveis', y=1.02, fontsize=16)
    plt.close(fig)
    return fig


def plot_countplot(
    dados: pd.DataFrame,
    coluna: str,
    titulo: str = "",
    figsize: tuple = (8, 5),
    palette: list = None,
):
    """
    Plota um countplot (gráfico de contagem) usando Seaborn.

    Args:
        dados (pd.DataFrame): DataFrame com os dados.
        coluna (str): Nome da coluna categórica a ser plotada.
        titulo (str, opcional): Título do gráfico.
        figsize (tuple, opcional): Tamanho da figura (largura, altura). Padrão (8,5).
        palette (list, opcional): Lista de cores personalizadas

    Returns:
        Figura com o gráfico.
    """
    plt.figure(figsize=figsize)

    ax = sns.countplot(
        data=dados,
        x=coluna,
        palette=palette
    )

    # Exibir valores acima das barras

    for container in ax.containers:
        ax.bar_label(container, fontsize=12)

    # Título e formatação
    plt.title(titulo, fontsize=16, fontweight="bold", loc="left")
    plt.xlabel(coluna, fontsize=13)
    plt.ylabel("Contagem", fontsize=13)

    plt.tight_layout()
    plt.show()

In [0]:
dados.head(2)

In [0]:
colunas_selecionadas = ['receive_time', 'valor_gasto_na_jornada', 'qtd_transacoes_na_jornada', 'ticket_medio_na_jornada', 'age', 'credit_card_limit', 'discount_value', 'duration', 'min_value', 'tempo_de_registro', 'num_channels']

In [0]:
plotar_outliers(dados[colunas_selecionadas])


Essas variáveis que apresentam outliers (`valor_gasto_na_jornada`, `qtd_transacoes_na_jornada` e `ticket_medio_na_jornada`) pela detecção usando o IQR, serão removidas posteriormente, na etapa de modelagem. 

Mas aqui servem para gente entender um pouco como é o perfil de consumo. Podemos tirar 3 principais conclusões:

 * Todas as 3 variáveis possuem forte assimetria à direita, ou seja, a maioria dos clientes tem valores baixos deixando a grande massa dos dados concentrada perto de 0. 
 
 * Poucos cliêntes têm valores muitos altos

 * Essa assimatria faz com que a média seja maior que a mediana

In [0]:
heatmap_corr(dados[colunas_selecionadas])

Nesse gráfico de correlação, há 3 relações que chamam a atenção:

  * `ticket_medio_na_jornada` e `valor_gasto_na_jornada` → 0.79
    → Faz sentido: quanto maior o gasto total, maior tende a ser o ticket médio.
  * `min_value` e `duration` → 0.47
    → Ofertas de maior valor mínimo tendem a ter maior duração.
  * `min_value` e `credit_card_limit` → 0.31
    → Usuários com maior limite de cartão tendem a receber/completar ofertas com valor mínimo mais alto.
  * `qtd_transacoes_na_jornada` e `valor_gasto_na_jornada` → 0.40
    → Mais transações aumentam o gasto total, de forma esperada.
  * `tempo_de_registro` tem correlação **negativa** com `min_value` (-0.32).
    → Clientes mais antigos tendem a lidar com ofertas de valor mínimo menor.

A correlação forte entre `valor_gasto_na_jornada` e `ticket_medio_na_jornada` sugere que talvez não seja necessário manter ambas no modelo de machine learning porque carregam informação semelhante. Mas como mencionado anteriormente, essas duas colunas vão ser removidas, portando não devemos nos preocupar com elas.

As demais correlações não são significativas. 


In [0]:
plot_countplot(dados, 'target', 'Distribuição target')

Portando a base de dados está com um bom balanceamento. Não sendo necessário aplicar técnicas para tratar isso. 

## 3. Desenvolvimento dos modelos 

Devemos primeiro definir qual métrica vamos maximizar, considerando a necessidade do negócio. Nosso objetivo é desenvolver um modelo que auxilie na decisão de qual oferta enviar para cada cliente. Entre as opções temos:

* Acurácia: Mede a proporção de previsões corretas, ou seja, tanto de clientes que aceitariam a oferta e dos que não aceitariam, em relação ao total de previsões. 

$$
\text{Acurácia} = \frac{TP + TN}{TP + TN + FP + FN}
$$

* Precisão (precision): Mede, de todas as vezes em que o modelo previu que um cliente aceitaria a oferta, quantas vezes de fato ele acertou. Uma alta precisão é importante em casos onde o custo do FALSO POSITIVO  é alto. Para o nosso contexto, um FALSO POSITIVO é um cliente que o modelo que disse aceitaria a oferta, mas na realidade não aceita. O custo real seria enviar uma oferta para um cliente que não vai converter. 

$$
\text{Precisão} = \frac{TP}{TP + FP}
$$

* Recall (sensibilidade): Mede, de todos os clientes que realmente aceitariam a oferta, quantos o modelo conseguiu identificar corretamente. Um alto recall é importante quando o custo de um FALSO NEGATIVO é alto. Um FALSO NEGATIVO para o nosso contexto seria um cliente que aceitaria a oferta, mas o modelo previu que não, e portanto a oferta não é enviada para ele. O custo seria a perda.

$$
\text{Recall} = \frac{TP}{TP + FN}
$$

* F1-Score: É a média harmônica entre Precisão e Recall. É útil quando é necessário ter um equilíbrio entre as duas. F1-Score somente é alto quando ambas as métricas forem altas também.  

$$
\text{F1-Score} = \frac{2 \cdot \text{Precisão} \cdot \text{Recall}}{\text{Precisão} + \text{Recall}}
$$

* AUC-ROC: A curva ROC mostra a capacidade do modelo de distinguir entre as classes positivas e negativa. A AUC é area sob essa curva que varia entre 0.5 e 1. Avalia o desempenho do modelo em todos os limiares possíveis. 

Pensando no problema, o F1-Score é a métrica que eu escolhi para maximizar que está alinhada com os objetivos do negócio. O F1 vai me dar um ótimo balanço entre não despedicar as ofertas (Precisão) e não perder oportunidades (Recall). Vamos mostrar todas as métricas, mas nosso foco em maximizar vai ser o **F1-SCORE**.


### Estrutura do projeto

#### Registrar o modelo usando MLflow

Quando registramos um modelo no Databricks com a ajuda do MLFlow, artefatos e metadatos importante são capturados. Isso garante que o modelo desenvolvido, não seja apenas capaz de ser reproduzido no ambiente, mas também que esteja completamente pronto para a implantação com as dependências necessárias.

![](https://www.databricks.com/wp-content/uploads/2020/04/databricks-adds-access-control-to-mlflow-model-registry_01.jpg)

#### Formulação do projeto


A formulação do projeto que vamos adotar é estimar a probabilidade de conversão para cada cliente por oferta e então escolher a oferta com maior probabilidade. Isso é mais útil que treinar um modelo que vai apenas dizer "O cliente vai aceitar a oferta".


A minha proposta é aplicar 2 técnicas de modelagem de *machine learning*, sendo elas:

* *Bagging*;
* *Boosting*.

Após a aplicação dessas técnicas, analisaremos qual apresentou o melhor desempenho de acordo com a métrica escolhida. 

Antes de passarmos para as técnicas, precisamos definir um modelo como *baseline* a ser ultrapassado. Vamos utilizar o `DecisionTreeClassifier` como nosso *baseline* já que todos os modelos que vamos utilizar serão baseados em árvores e `DecisionTreeClassifier` seria o mais simples de todos. 



#### Bagging - RandomForestClassifier

Bagging vem de *Bootstrap Aggregating*:

* Bootstrap = criar várias amostras diferentes dos dados originais (com reposição).

* Aggregating = juntar as previsões de todos os modelos (por média ou votação)


**Funcionamento:**

1. Temos o conjunto de dados inicial.

2. Gera diversas amostras aleatórias com reposição a partir desse conjunto de dados.

3. Para cada amostra aleatória gerada, treina um modelo diferente (uma árvore de decisão no nosso caso).

4. Ao final, junta os resultados de todos os modelos. Como é uma classificação, o resultado será a votação da maioria. 

![](https://i.imgur.com/EGDJjvk.png)

#### Boosting - XGBoost

Boosting é uma técnica que vai aprendendo com erros dos preditores anteriores, ou seja, modelos simples que são treinados em sequência, onde cada um tenta corrigir os erros dos anteriores. 

**Funcionamento:**

1. Temos a base de dados inicial
2. Treinamos o primeiro modelo
3. Avaliamos onde ele errou
4. Dá mais peso para os exemplos onde ele errou
5. Treina o segundo modelo, porém focando em corrigir esses erros
6. Repete o processo, até o total de modelos indicados (no nosso caso o número de árvores de decisão)
7. Combina todas as previsões (Geralmente usando uma média ponderada ou soma com pesos)


![](https://i.imgur.com/N4IRnZP.png)

Para a nosa implementação usaremos o [XGBoost](https://xgboost.readthedocs.io/en/stable/).

### Implementando a estrutura

In [0]:
X = dados.drop(columns=['target'])
y = dados['target']

In [0]:
X.columns

In [0]:
num_cols = ['receive_time', 'age', 'credit_card_limit',
        'tempo_de_registro', 'discount_value', 'duration',
       'min_value', 'num_channels', 'email',
       'mobile', 'social', 'web']

cat_cols = ['gender', 'offer_type', 'categoria_duracao']

cols_drop = ['account_id', 'offer_id_final', 'offer_received', 'offer_viewed',
    'valor_gasto_na_jornada', 'qtd_transacoes_na_jornada',
    'ticket_medio_na_jornada', 'registered_on', 'channels']


Removemos `account_id` e `offer_id_final` pois são identificadores. `registered_on` é uma data, e já temos o `tempo_de_registro` como uma variável numérica.

O motivo de remover `valor_gasto_na_jornada`, `qtd_transacoes_na_jornada` e `ticket_medio_na_jornada` é um pouco mais interessante. Essas variáveis contém informações sobre o resultado que estamos tentando prever, mas que só estariam disponíveis depois do evento acontecer. Nosso objetivo é prever se um cliente vai completar a oferta antes de enviá-la. O valor que o cliente gastou após visualizar a oferta ou mesmo depois de receber poderia dar dicas ao modelo que não queremos. Seria algo como "Se o cliente gastou muito após visualizar a oferta, preveja que ele vai completar". Mas no momento do envio da oferta não sabemos se ele vai gastar muito ou pouco. 

`offer_received` e `offer_viewed` são as variáveis da "história" do cliente, como mencionado no notebook 1. Essas variáveis indicam os estágios da conversão. `offer_received` sempre vai ser 1 para todas as linhas. Elas também não ajudam na decisão principal. A decisão de "qual oferta enviar" acontece antes mesmo de sabermos se o cliente irá sequer visualizá-la. Incluir a coluna `offer_viewed` pode confundir o modelo. Queremos prever o sucesso da oferta do cliente, e as características do cliente e da oferta são as causas que levam a esse sucesso.`channels` é uma variável que já foi transformada no notebook 01. Foi mantida apenas para ser utilizada no Dashboard Power Bi.


### Pipeline de ML

O pipeline de ML abaixo está englobando algumas etapas do processo de modelagem, como:

* Treinamento do modelo de machine learning. Nesse caso estamos testando 3 modelos. Decision Tree, Random Forest e XGBoost.

* Otimização dos hiperparâmetros desses modelos para decidir qual técnica/modelo teve melhor desempenho. Para avaliar com mais precisão utilizamos a validação cruzada. 

* Registro no MLFlow dos 3 melhores modelos otimizados.

* Salvar os 3 melhores modelos no Unity Catalog utilizando o MLFlow.

In [0]:
def espaco_busca_dt(trial):
    # Espaço de busca para árvore de decisão

    return {
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'max_depth': trial.suggest_int('max_depth', 5, 30),
        'random_state': 42

        # Demais hiperâmetros que serão otimidos por questão de velocidade do código. 

    }

def espaco_busca_rf(trial):
    # Espaço de busca para o random forest

    return {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'max_depth': trial.suggest_int('max_depth', 5, 30),
        'random_state': 42

        # Demais hiperâmetros que serão otimidos por questão de velocidade do código. 
    }

def espaco_busca_xgb(trial):
    # Espaço de busca para o XGBoost

    return {
        'n_estimators': trial.suggest_int('n_estimators', 100, 400, step=50),
        'max_depth': trial.suggest_int('max_depth', 5, 30),
        'random_state': 42

        # Demais hiperâmetros que serão otimidos por questão de velocidade do código. 
    }

In [0]:
def objective(trial, classe_modelo, espaco_busca, X_train, y_train, X_test, y_test, cat_cols, num_cols):

    # Diz ao Optuna como avaliar se uma combinação de parâmetros é boa ou ruim.

    with mlflow.start_run(nested=True):
        skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        
        params = espaco_busca(trial)
        mlflow.log_params(params)
        
        model = classe_modelo(**params)
        
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', 'passthrough', num_cols),
                ('cat', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False), cat_cols)
            ],
            remainder='passthrough'
        )
        
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', model)
        ])

        # Validação cruzada
        scores = cross_val_score(pipeline, X_train, y_train, cv=skf, scoring='f1')

        # Valor a ser maximizado na otimização de hiperparâmetros
        mean_f1 = np.mean(scores)

    return mean_f1

In [0]:
def otimizar_e_registrar_modelo(
    X, y, colunas_para_dropar, cat_cols, num_cols, 
    nome_modelo, classe_modelo, espaco_busca_modelo, 
    n_trials=10
):

    X_model = X.drop(columns=colunas_para_dropar)
    X_model[num_cols] = X_model[num_cols].astype("float64")
    X_train, X_test, y_train, y_test = train_test_split(X_model, y, random_state=42, stratify=y)
    
    TZ_MINUS3 = timezone(timedelta(hours=-3))
    timestamp = datetime.now(TZ_MINUS3).strftime("%Y-%m-%d_%H-%M-%S")
    run_name = f"Otimizacao_{nome_modelo}_{timestamp}"

    with mlflow.start_run(run_name=run_name) as parent_run:
        print(f"--- Iniciando otimização para o modelo: {nome_modelo} ---")
        mlflow.set_tag("model_type", nome_modelo)

        objective_com_dados = partial(
            objective,
            classe_modelo=classe_modelo,
            espaco_busca=espaco_busca_modelo,
            X_train=X_train, y_train=y_train,
            X_test=X_test, y_test=y_test,
            cat_cols=cat_cols, num_cols=num_cols
        )
        
        study = optuna.create_study(direction="maximize")
        study.optimize(objective_com_dados, n_trials=n_trials)
        
        mlflow.log_params(study.best_params)

        best_model = classe_modelo(**study.best_params)
        
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', 'passthrough', num_cols),
                ('cat', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False), cat_cols)
            ]
        )
        best_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', best_model)])
        best_pipeline.fit(X_train, y_train)

        y_pred = best_pipeline.predict(X_test)
        y_pred_proba = best_pipeline.predict_proba(X_test)[:, 1]
        metrics = {
            'Acurácia': accuracy_score(y_test, y_pred),
            'Precisão': precision_score(y_test, y_pred),
            'Recall': recall_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred)
        }

        print("Gerando e registrando visualizações...")
        
        labels = ["True Neg","False Pos","False Neg","True Pos"]
        categories = ["Não completada", "Completada"]
        
        fig_cm = plot_matriz_confusao(
            y_test, 
            y_pred, 
            group_names=labels,
            categories=categories,
            title=f"Matriz de Confusão para {best_model.__class__.__name__}",
            figsize=(15, 5)
        )
        mlflow.log_figure(fig_cm, "matriz_de_confusao.png")
        plt.close(fig_cm)
        
        fig_roc = plot_roc_auc(y_test, y_pred_proba)
        mlflow.log_figure(fig_roc, "curva_roc_auc.png")
        plt.close(fig_roc)

        print(f"Registrando o modelo '{nome_modelo}' no MLflow...")

        # Modelo e Avaliação Automática
        signature = infer_signature(X_train, y_pred)

        local_salvo = f"workspace.modelos_ml.{nome_modelo}"

        model_info = mlflow.sklearn.log_model(
            sk_model=best_pipeline,
            artifact_path="modelo",
            signature=signature,
            registered_model_name=local_salvo
        )

        evaluation_data = X_test.copy()
        evaluation_data["target"] = y_test
        mlflow.models.evaluate(
            model=model_info.model_uri,
            data=evaluation_data,
            targets="target",
            model_type="classifier"
        )
        
        print(f"Modelo '{nome_modelo}' otimizado e registrado com sucesso!")
        return local_salvo, metrics

In [0]:
MODEL_CONFIG = {
    "ArvoreDecisao": {
        "classe_modelo": DecisionTreeClassifier,
        "espaco_busca_modelo": espaco_busca_dt
    },
    "RandomForest": {
        "classe_modelo": RandomForestClassifier,
        "espaco_busca_modelo": espaco_busca_rf
    },
    "XGBoost": {
        "classe_modelo": XGBClassifier,
        "espaco_busca_modelo": espaco_busca_xgb
    }
}

resultados = []

for model_name, config in MODEL_CONFIG.items():
    
    # Otimiza o modelo atual
    local_salvo, info_modelo = otimizar_e_registrar_modelo(
        X=X, 
        y=y,
        colunas_para_dropar=cols_drop,
        cat_cols=cat_cols,
        num_cols=num_cols,
        nome_modelo=f"ModeloOferta_{model_name}",
        classe_modelo=config['classe_modelo'],
        espaco_busca_modelo=config['espaco_busca_modelo'],
        n_trials=10 
    )
    
    info_modelo['caminho'] = local_salvo
    resultados.append(info_modelo)

results_df = pd.DataFrame(resultados).sort_values(by='f1_score', ascending=False).reset_index(drop=True)

In [0]:
print("--- Comparativo Final dos Modelos Otimizados ---")
results_df

Olhando para o *f1_score* os 3 tiveram praticamente um empate técnico. Porém, os Falsos Negativos do modelo Random Forest foram menores e, pensando no negócio, considero que os Falsos Negativos que representam um cliente que aceitaria a oferta, mas o modelo previu que não, são mais prejudiciais do que os Falsos Positivos. Portanto visando maximizar ganhos, vamos escolher o modelo Random Forest. Poderíamos ter tido as mesmas conclusões olhando para o **recall** como métrica secundária. Para garantir que quem tá executando o código consiga ver a imagem abaixo, vai ser disponilizado em markdown as imagens da matrizes de confusão, mas na entrevista pretendo mostrar no mlflow a comparação. 

**Matriz de confusão para decision tree**
![](https://i.imgur.com/QvNGiJc.png)

**Matriz de confusão random forest**
![](https://i.imgur.com/H5fqqIW.png)

**Matriz de confusão XGBoost**
![](https://i.imgur.com/0z7gO3X.png)


### Prevendo qual oferta enviar para determinado cliente

In [0]:
client = MlflowClient()
versao_modelo = 1
client.set_registered_model_alias("workspace.modelos_ml.ModeloOferta_RandomForest", "producao", versao_modelo)

In [0]:
dados.columns

In [0]:
model_uri = "models:/workspace.modelos_ml.ModeloOferta_RandomForest@producao"

print(f"Carregando modelo: {model_uri}")
load_model = mlflow.sklearn.load_model(model_uri)
print("Modelo carregado!")

base = X.drop(columns=cols_drop, errors="ignore")
df_cliente = (
    base.loc[:, ['age', 'credit_card_limit', 'gender', 'tempo_de_registro', 'receive_time']]
        .sample(1)
        .reset_index(drop=True)
)

offer_features = [
    'offer_id_final', 'discount_value', 'duration', 'min_value', 'offer_type',
    'categoria_duracao', 'num_channels', 'email', 'mobile', 'social', 'web'
]
offer_features = list(dict.fromkeys(offer_features))  # garante unicidade mantendo ordem
dados_ofertas = dados.loc[:, offer_features].drop_duplicates().reset_index(drop=True)

df_scoring = df_cliente.merge(dados_ofertas, how='cross')

colunas_modelo = base.columns.tolist()
df_scoring = df_scoring.reindex(columns=colunas_modelo, fill_value=0)

print("Dados do cliente: ")
display(df_cliente)

print("Calculando a probabilidade de aceite para cada oferta...")
probabilidade_aceite = load_model.predict_proba(df_scoring)[:, 1] * 100

df_proba_sorted = (
    dados_ofertas.assign(probabilidade_aceite=probabilidade_aceite.round(2))
                 .sort_values('probabilidade_aceite', ascending=False)
                 .reset_index(drop=True)
)

display(df_proba_sorted)


### Extra (Mostrar servindo o modelo)